In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_books = pd.read_csv('/content/drive/MyDrive/Capstone Project (Books Reviews)/Data/goodreads_books.csv')
df_books.head(3)

,isbn,series,language_code,average_rating,description,authors,publication_year,image_url,book_id,work_id,title
0,0312853122,[],NaN,4.00,NaN,"[{'author_id': '604031', 'role': ''}]",1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,5400751.0,W.C. Fields: A Life on Film
1,0743509986,[],NaN,3.23,"Anita Diamant's international bestseller ""The ...","[{'author_id': '626222', 'role': ''}]",2001.0,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,1323437.0,Good Harbor
2,NaN,['189911'],eng,4.03,Omnibus book club edition containing the Ladie...,"[{'author_id': '10333', 'role': ''}]",1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."


In [ ]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360655 entries, 0 to 2360654
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   isbn              object 
 1   series            object 
 2   language_code     object 
 3   average_rating    float64
 4   description       object 
 5   authors           object 
 6   publication_year  float64
 7   image_url         object 
 8   book_id           int64  
 9   work_id           float64
 10  title             object 
dtypes: float64(3), int64(1), object(7)
memory usage: 198.1+ MB


In [ ]:
# Encoding ID columns to continuous ids.
def proc_col(col, train_col=None):
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

def encode_data(df, cols, train=None):
    df = df.copy()
    for col_name in cols:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [ ]:
df_books1 = encode_data(df_books,['work_id'])
df_books1 = encode_data(df_books,['book_id'])
df_books1['authors'] = df_books1['authors'].apply(lambda x: [i['author_id'] for i in eval(x)])
df_books1['series'] = df_books1['series'].apply(eval)

In [ ]:
df_genre = pd.read_csv('/content/drive/MyDrive/Capstone Project (Books Reviews)/Data/genres.csv', index_col = 0)
df_genre1 = encode_data(df_genre,['book_id'], df_books)
df_genre1 = df_genre1.merge(df_books1[['book_id','work_id']], how = 'left', on = 'book_id')
df_books1 = df_books1.merge(df_genre1[['work_id','genres_new']], how = 'left', on ='work_id')
del df_genre, df_genre1

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_reviews = pd.read_csv('/content/drive/MyDrive/Capstone Project (Books Reviews)/Data/goodreads_review_spoiler.csv')
df_reviews = df_reviews.merge(df_books[['work_id','book_id','title']], how = 'left', on = 'book_id')
df_reviews1 = encode_data(df_reviews,['work_id'], df_books)
df_reviews1 = encode_data(df_reviews, ['user_id'])
df_reviews1.drop(['timestamp','review_id'],axis=1, inplace=True)
df_reviews1.dropna(subset = ['work_id'], inplace=True)

df_reviews1['work_id'] = df_reviews1['work_id'].astype(int)
df_reviews1['rating'] = pd.to_numeric(df_reviews1['rating'], downcast ='integer')
df_reviews1['has_spoiler'] = pd.to_numeric(df_reviews1['has_spoiler'], downcast ='integer')
df_reviews1.head()

,user_id,book_id,rating,has_spoiler,review_sentences,work_id,title
0,0,18245960,5,1,"[[0, 'This is a special book.'], [0, 'It start...",25696480,The Three-Body Problem (Remembrance of Earth’s...
1,0,16981,3,0,"[[0, 'Recommended by Don Katz.'], [0, 'Avail f...",170957,Invisible Man
2,0,28684704,3,1,"[[0, 'A fun, fast paced science fiction thrill...",43161998,Dark Matter
3,0,27161156,0,0,"[[0, 'Recommended reading to understand what i...",47200486,Hillbilly Elegy: A Memoir of a Family and Cult...
4,0,25884323,4,1,"[[0, 'I really enjoyed this book, and there is...",42742263,Aurora


In [ ]:
pd.DataFrame(df_reviews1['book_id'].unique(), columns = ['book_id']).merge(pd.DataFrame(df_books1[['book_id', 'language_code']]), how = 'left', on='book_id')['language_code'].value_counts()

eng      9684
spa      2498
ita      1540
en-US     987
fre       906
ger       822
por       654
ind       652
en-GB     647
fin       454
ara       267
nl        246
slo       244
cze       223
pol       217
gre       205
rus       189
tur       175
vie       146
rum       143
scr       137
hun       129
swe       107
bul       107
tha        96
dan        94
cat        91
per        82
isl        82
est        80
pt-BR      71
srp        71
en-CA      64
aze        59
lav        52
lit        36
jpn        29
heb        26
ben        10
slv         9
nob         7
msa         7
kat         6
mul         5
mon         3
zho         3
ukr         2
afr         2
nor         2
mya         1
urd         1
tam         1
mal         1
wel         1
en          1
Name: language_code, dtype: int64

In [ ]:
df_books1['language_code'].value_counts()

eng      5211066
spa      1014395
ita       761699
en-US     556272
por       465950
          ...   
chv            1
chi            1
afa            1
myv            1
lao            1
Name: language_code, Length: 226, dtype: int64

In [ ]:
df_books1['language_code'].unique()

array([nan, 'eng', 'ger', 'spa', 'en-US', 'ita', 'per', 'en-GB', 'tur',
       'ind', 'mon', 'fre', 'por', 'ara', 'en-CA', 'tha', 'lav', 'jpn',
       'pol', 'swe', 'kor', 'fin', 'msa', 'bul', 'nl', 'gre', 'slo',
       'nor', 'heb', 'hun', 'ben', 'scr', 'zho', 'fil', 'rus', 'lit',
       'rum', 'cze', 'dan', 'slv', 'nno', 'pes', 'hye', 'nob', 'cat',
       'en', 'vie', 'nep', 'mar', 'srp', 'urd', 'guj', 'est', 'sqi',
       'ukr', 'afr', 'mul', 'grc', 'kat', 'mkd', 'hin', 'tam', 'mus',
       '--', 'bos', 'enm', 'gla', 'isl', 'glg', 'mal', 'kur', 'wel',
       'pt-BR', 'crh', 'tel', 'es-MX', 'kan', 'mya', 'fao', 'aze', 'ota',
       'arw', 'pra', 'tgl', 'lat', 'dum', 'eus', 'sin', 'mlt', 'ada',
       'apa', 'udm', 'peo', 'bel', 'iro', 'nld', 'ori', 'smn', 'amh',
       'tut', 'frs', 'arg', 'ang', 'abk', 'epo', 'snd', 'pan', 'egy',
       'dut', 'vls', 'jav', 'tlh', 'din', 'gle', 'alg', 'gsw', 'nah',
       'her', 'aus', 'aka', 'chm', 'ace', 'oci', 'ast', 'kok', 'tib',
       'frm', '

To reduce complexity, we will rule out non-English books and their reviews. Also, we will consider the books regardless the editions. We will use work_id over book_id.

In [ ]:
language = ['eng', 'en-US', 'en-GB', 'en-CA', 'en']
df_books1 = df_books1[(df_books1['language_code'].isin(language))|(df_books1['language_code'].isnull())]

In [ ]:
df_reviews1 = df_reviews1.merge(df_books1[['language_code','book_id']], how = 'left', on = 'book_id')
df_reviews1 = df_reviews1[(df_reviews1['language_code'].isin(language))|(df_reviews1['language_code'].isnull())]

In [ ]:
df_books1 = df_books1.sort_values('language_code').drop_duplicates(subset='work_id').reset_index().drop('index',axis=1)

In [ ]:
df_books1[['genres_new','description','title']].isna().sum()

genres_new     1268665
description     299963
title                9
dtype: int64

In [ ]:
df_books1 = df_books1.dropna(axis = 0, subset=['title','work_id'])
df_books1['genres_new'] = df_books1['genres_new'].fillna('')
df_books1['description'] = df_books1['description'].fillna('')
df_books1['work_id'] = df_books1['work_id'].astype(int)

In [ ]:
import pickle 
pickle.dump(df_books1,open('/content/drive/MyDrive/Capstone Project (Books Reviews)/df_books.pickle','wb'))
pickle.dump(df_reviews1,open('/content/drive/MyDrive/Capstone Project (Books Reviews)/df_reviews.pickle','wb'))